# 🧑‍💻 Amazon Nova built-in Code Interpreter System Tool

Amazon Nova 2 comes with a built-in code interpreter that allows Amazon Nova to execute code.

The Code Interpreter allows Amazon Nova to generate code that get's executed in a secure sandbox. 
This is useful for problems that require math, code writing or code review, and data analysis.

Example Use Cases:
- Data Analysis, Data Visualization, Data-Driven Decision-Making
- Code Assistant, Code Reviewer
- Math
- Other use cases that require reproducible computation rather than probablistic next token prediction

When not to use Code Interpreter:
- If you need to retrieve real-time information from the web. For such use cases we recommend to use the Amazon Nova web search system tool.
- Also currently, Amazon Nova Code Interpreter does not have access to your AWS resources. So for automation tasks or to get and upload data to an Amazon S3 bucket or database we recommend to user Amazon Bedrock AgentCore.

This notebook guides you how to use the Code Interpreter.

## 🎯 What You'll Build

By the end of this workshop, you will:
- ✅ Understand how the API for Code Interpreter works
- ✅ Know how to use Code Interpreter
- ✅ Best practices for using Code Interpreter

## Architecture of Code Interpreter

![](./img/Code-Interpreter.png)

## Prerequisites
- AWS Account
- [IAM Permissions](https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html) or an Amazon Bedrock [API Key](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_bedrock.html) to invoke Amazon Nova 2
- Python 3.8+

## Workshop Outline
1. Setup and Configuration
2. Quick start with Nova Code Interpreter
3. Sample Use Cases that require Code execution
4. Best Practices for Code Intepreter
5. Next Steps

## 1. Setup and Configuration

This section covers the initial setup required to use Amazon Nova's Code Interpreter.
We'll configure the AWS SDK (boto3) with appropriate settings for code execution tasks.

In [3]:
%pip install --quiet ./onboarding_materials/boto3-1.40.26-py3-none-any.whl ./onboarding_materials/botocore-1.40.26-py3-none-any.whl

Note: you may need to restart the kernel to use updated packages.


Before running this script, ensure you have the required dependencies installed:

In [ ]:
#%pip install --quiet "boto3>=x.x.x." "botocore>=x.x.x." TODO after release

In [ ]:
from IPython.display import Markdown, JSON

import boto3
from botocore.config import Config
import json 

The `create_boto3_client` helper function initializes a boto3 client for Amazon Bedrock Runtime
with custom timeout configurations. These extended timeouts are crucial for Code Interpreter
tasks that may take longer to execute.

In [ ]:
def create_boto3_client():
    client = boto3.client(
        "bedrock-runtime",
        region_name="us-west-2",
        config=Config(
            connect_timeout=3600,  # 60 minutes code interpreter might be more long running
            read_timeout=3600,     # 60 minutes
            retries={'max_attempts': 1}
        )
    )
    return client

In [ ]:
bedrock_client = create_boto3_client()

In [6]:
NOVA_2_LITE = "us.amazon.nova-2-lite-omni-v1:0"

## 2. Quick start with Nova Code Interpreter

### Converse API

This section demonstrates how to use Amazon Nova's Code Interpreter through the Converse API.
The Converse API provides a unified interface for interacting with foundation models,
including support for system tools like the Code Interpreter.

The `construct_convers_payload` helper function constructs the payload required for the Converse API.
It includes the model ID, messages, and other parameters like temperature, max tokens, and tool configurations.

In [7]:
def construct_convers_payload(model, messages, temperature, max_tokens, top_p = 0.9, reasoning_effort = None, tools = None):
    inferenceConfig = {
            "maxTokens": max_tokens,
    }

    if temperature:
        inferenceConfig["temperature"] = temperature

    if top_p:
        inferenceConfig["topP"] = top_p

    tool_config = None
    if tools:
        tool_config = {
            "tools": tools
          }

    additional_request_fields = {}
    if reasoning_effort:
        reasoning_config = {
          "type": "enabled",
          "maxReasoningEffort": reasoning_effort
        }
        additional_request_fields["reasoningConfig"] = reasoning_config

    request = {
        "modelId":model, 
        "messages":messages, 
        "inferenceConfig":inferenceConfig,
        "toolConfig":tool_config,
        "additionalModelRequestFields":additional_request_fields
    }
    return request

The `amazon_bedrock_nova_converse` helper function is a wrapper that:
1. Constructs the request payload
2. Calls the Bedrock Runtime converse() API
3. Extracts the text response from the model
4. Returns both the text output and full response object

In [8]:
def amazon_bedrock_nova_converse(model, messages, temperature, max_tokens, top_p = 0.9, reasoning_effort = None, tools = None, bedrock_rt_client = None ):  
    if not bedrock_rt_client:
        bedrock_rt_client = create_boto3_client()

    try:
        request = construct_convers_payload(model, messages, temperature, max_tokens, top_p, reasoning_effort, tools)

        model_response = bedrock_rt_client.converse(**request)
        
        output = model_response["output"]["message"]["content"][-1]["text"]
        return output, model_response
        
    except Exception as e:
        print(type(e), e)
        return str(e), e

    

This example demonstrates the basic workflow of using Code Interpreter with the converse API. To give Amazon Nova access to the Code Interpreter you specify the `systemTool` parameter in the tools config. 

In [137]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Print hello world using the code interpreter"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

#### Understanding the Response

The model response contains multiple layers of information. Let's examine each part.
Here is the final output from Amazon Nova:

In [ ]:
print("Final response from Amazon Nova:")
print(output)

'The code has been executed, and it printed:\n\n**Hello World**'

In addition to the final output, you can examine the model response to understand
what code was executed in the Code Interpreter. This is valuable for:
- Debugging: See exactly what code was generated
- Learning: Understand how the model approaches problems
- Auditing: Track what code is being executed in your application

In [130]:
content = model_response['output']['message']['content']

Extract tool use and tool result from the response content:

In [131]:
tool_use = next((trace for trace in content if "toolUse" in trace), None)
tool_result = next((trace for trace in content if "toolResult" in trace), None)

The `toolUse` block shows what code Amazon Nova generated and sent to the Code Interpreter. The `toolUse` > `input` > `code` property contains the code that got executed in the Code Interpreter.

In [134]:
JSON(tool_use, expanded=True)

<IPython.core.display.JSON object>

The output of the Code Interpreter tool contains the logs from `stdOut` in the Code Interpreter environment and also any errors that occured in the Code Interpreter environment.

In [136]:
JSON(tool_result, expanded=True)

<IPython.core.display.JSON object>

### Converse API with Streaming

Code Interpreter also works with streaming the response. Streaming allows you to receive the model's response incrementally as it's generated, rather than waiting for the complete response.

The `process_stream` helper function handles receiving the stream of output blocks.

In [ ]:
def process_stream(stream_response):
    is_reasoning = False
    is_text = False
    is_toolUse = False
    is_tool_result = False
    model_output = ""  # Initialize text collector
    
    for message in stream_response["stream"]:
        if "contentBlockStart" in message:
            if "toolUse" in message["contentBlockStart"]["start"]:
                is_toolUse = True
                is_text = False
                is_reasoning = False
                print("\n\n----MODEL RESPONSE: TOOL USE-----\n")
                tool_use = message["contentBlockStart"]["start"]["toolUse"]
                print(tool_use["name"])

        if "contentBlockDelta" in message:
            delta = message["contentBlockDelta"]["delta"]

            # Check for regular text
            if "text" in delta:
                if not is_text:
                    print("\n\n----MODEL RESPONSE: TEXT-----\n")
                    is_text = True
                    is_reasoning = False
                    is_toolUse = False
                text_chunk = delta["text"]
                model_output += text_chunk
                print(text_chunk, end="", flush=True)

            # Check for reasoning content
            elif "reasoningContent" in delta and "text" in delta["reasoningContent"]:
                if not is_reasoning:
                    print("\n\n----MODEL RESPONSE: REASONING-----\n")
                    is_text = False
                    is_reasoning = True
                    is_toolUse = False
                print(delta["reasoningContent"]["text"], end="", flush=True)

            elif "toolUse" in delta:
                print(delta)
            elif "toolResult" in delta:
                if not is_tool_result:
                    print("\n\n----MODEL RESPONSE: TOOL RESULT-----\n")
                    is_tool_result = True
                    is_toolUse = False
                print(delta)
        elif "messageStop" in message:
            print("\n\n----MODEL RESPONSE: messageStop-----\n")
            print(json.dumps(message, indent=4))
        elif "metadata" in message:
            print("\n\n----MODEL RESPONSE: metadata-----\n")
            print(json.dumps(message, indent=4))
    
    return model_output


Similar to the non-streaming version, but calls `converse_stream` instead. The function returns a stream object that must be iterated to receive events.

In [10]:
def amazon_bedrock_nova_converse_streaming(model, messages, temperature, max_tokens, top_p = 0.9, reasoning_effort = None, tools = None, bedrock_rt_client = None ):  
    if not bedrock_rt_client:
        bedrock_rt_client = create_boto3_client()

    try:
        request = construct_convers_payload(model, messages, temperature, max_tokens, top_p, reasoning_effort, tools)

        model_response = bedrock_rt_client.converse_stream(**request)
        
        return model_response
        
    except Exception as e:
        print(type(e), e)
        return e

    

Here is an example using the response streaming:

In [139]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Calculate 7^6 using the code interpreter"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

response = amazon_bedrock_nova_converse_streaming(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
model_output = process_stream(response)



----MODEL RESPONSE: TOOL USE-----

nova_code_interpreter
{'toolUse': {'input': '{"code":"\\"\\"\\"\\nCalculate 7^6 using Python\\n\\"\\"\\"\\n7**6"}'}}


----MODEL RESPONSE: TOOL RESULT-----

{'toolResult': [{'text': '{"stdOut":"117649","stdErr":"","exitCode":0,"isError":false}'}]}


----MODEL RESPONSE: TEXT-----

The result of \(7^6\) is **117,649**.

----MODEL RESPONSE: messageStop-----

{
    "messageStop": {
        "stopReason": "end_turn"
    }
}


----MODEL RESPONSE: metadata-----

{
    "metadata": {
        "usage": {
            "inputTokens": 945,
            "outputTokens": 61,
            "totalTokens": 1006
        },
        "metrics": {
            "latencyMs": 2672
        }
    }
}


The final response from Amazon Nova is:

In [140]:
Markdown(model_output)

The result of \(7^6\) is **117,649**.

### 3. Sample Use Cases that require Code execution

This section demonstrates real-world scenarios where Code Interpreter excels. These examples showcase the power of combining LLM reasoning with code execution for tasks that require precise computation.

#### Solving Polynomial Equations

Polynomial equations often have complex or multiple roots that are difficult to find even for reasoning models. Code Interpreter can use numerical methods (like NumPy's roots function) to find all solutions, including complex roots.

In [75]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Use the code interpreter to solve for all roots of the polynomial equation: z^6 + z^4 + z^3 + z^2 + 1 = 0"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 10000
reasoning_effort = None

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

Here is the solution for the polynomial:

In [74]:
Markdown(output)

The polynomial factors nicely:

\[
z^6 + z^4 + z^3 + z^2 + 1 = (z^{2} - z + 1)(z^{4} + z^{3} + z^{2} + z + 1).
\]

Thus the roots are the roots of the two factors.

---

### 1. Roots of \(z^{2} - z + 1 = 0\)

Solve the quadratic:

\[
z = \frac{1 \pm \sqrt{1 - 4}}{2} = \frac{1 \pm \sqrt{-3}}{2} = \frac{1 \pm i\sqrt{3}}{2}.
\]

These are two complex primitive 6‑th roots of unity, often denoted \(\omega\) and \(\omega^5\) where \(\omega = e^{i\pi/3}\). Explicitly:

\[
z_{1,2}= \frac{1}{2} \pm i\frac{\sqrt{3}}{2}= e^{\pm i\pi/3}.
\]

---

### 2. Roots of \(z^{4} + z^{3} + z^{2} + z + 1 = 0\)

This quartic is the **5th cyclotomic polynomial** \(\Phi_{5}(z)\). Its roots are the primitive 5‑th roots of unity, i.e. the complex numbers \(e^{2\pi i k/5}\) for \(k = 1,2,3,4\) (excluding \(k=0\) which gives 1).

Explicitly, using \(\zeta = e^{2\pi i/5} = \cos(2\pi/5) + i\sin(2\pi/5)\),

\[
\begin{aligned}
z_{3} &= e^{2\pi i/5}= \cos\frac{2\pi}{5} + i\sin\frac{2\pi}{5},\\[2mm]
z_{4} &= e^{4\pi i/5}= \cos\frac{4\pi}{5} + i\sin\frac{4\pi}{5},\\[2mm]
z_{5} &= e^{6\pi i/5}= \cos\frac{6\pi}{5} + i\sin\frac{6\pi}{5}
      = \cos\frac{4\pi}{5} - i\sin\frac{4\pi}{5},\\[2mm]
z_{6} &= e^{8\pi i/5}= \cos\frac{8\pi}{5} + i\sin\frac{8\pi}{5}
      = \cos\frac{2\pi}{5} - i\sin\frac{2\pi}{5}.
\end{aligned}
\]

The exact algebraic expressions for the cosine and sine values are:

\[
\cos\frac{2\pi}{5}= \frac{-1+\sqrt{5}}{4},\qquad 
\sin\frac{2\pi}{5}= \sqrt{\frac{5+\sqrt{20}}{8}}.
\]

Thus the four quartic roots can be written as:

\[
\begin{aligned}
z_{3,4} &= \frac{-1+\sqrt{5}}{4} \pm i\sqrt{\frac{5+\sqrt{20}}{8}},\\[2mm]
z_{5,6} &= \frac{-1-\sqrt{5}}{4} \pm i\sqrt{\frac{5-\sqrt{20}}{8}}.
\end{aligned}
\]

(Any equivalent exact forms are acceptable.)

---

## Final answer

All six roots of  

\[
z^{6} + z^{4} + z^{3} + z^{2} + 1 = 0
\]

are

\[
\boxed{
\begin{aligned}
&z_{1}= \frac{1}{2} + i\frac{\sqrt{3}}{2},\\
&z_{2}= \frac{1}{2} - i\frac{\sqrt{3}}{2},\\
&z_{3}= \frac{-1+\sqrt{5}}{4} + i\sqrt{\frac{5+\sqrt{20}}{8}},\\
&z_{4}= \frac{-1+\sqrt{5}}{4} - i\sqrt{\frac{5+\sqrt{20}}{8}},\\
&z_{5}= \frac{-1-\sqrt{5}}{4} + i\sqrt{\frac{5-\sqrt{20}}{8}},\\
&z_{6}= \frac{-1-\sqrt{5}}{4} - i\sqrt{\frac{5-\sqrt{20}}{8}}.
\end{aligned}
}
\]

Equivalently, in exponential form:

\[
z = e^{\pm i\pi/3},\; e^{2\pi i k/5}\ (k=1,2,3,4).
\]

#### Statistical Hypothesis Testing

Statistical analysis is a perfect use case for Code Interpreter because it requires:
- Precise mathematical calculations
- Standard statistical libraries (scipy, numpy)
- Amazon Nova's capability to interpret the results in context

In [92]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": """I have two datasets of student test scores. Perform a t-test to determine if there's a statistically significant difference between them at α=0.05 confidence level.

Group A: [78, 85, 92, 88, 79, 91, 84, 87, 90, 82]
Group B: [72, 75, 81, 69, 74, 78, 71, 76, 73, 70]
"""
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 10000
reasoning_effort = None

output, model_response = amazon_bedrock_nova_converse(NOVA_2_LITE, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

Here is the output of the statistical analysis that Amazon Nova conducted using the code interpreter:

In [ ]:
print("\nStatistical analysis results:")
Markdown(output)

**Results of the t-test**

- **t-statistic:** ≈ 6.03  
- **p-value:** ≈ 1.07 × 10⁻⁵  

**Interpretation**

The p-value (1.07 × 10⁻⁵) is far smaller than the significance level α = 0.05. Therefore, we **reject the null hypothesis** that the two groups have equal means.

**Conclusion**

There is a statistically significant difference between the test scores of Group A and Group B at the 95 % confidence level (α = 0.05). Group A scores are higher on average than Group B.

## 4. Best Practices for Code Intepreter

TODO

## 5. Next Steps

TODO